In [1]:
import numpy as np
import pandas as pd
from sys import executable, argv
from subprocess import check_output
import pctils as pc

import numpy as np
import pandas as pd
from sys import executable, argv
from subprocess import check_output
import pctils as pc
# SKLearn related imports
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import KNeighborsClassifier
import spacy
import hashlib

from tqdm import tqdm
from collections import Counter
from spacy.matcher import Matcher
from sklearn.metrics import accuracy_score
from nltk.tokenize import WordPunctTokenizer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.preprocessing import OneHotEncoder


In [2]:
import nltk
nltk.download('stopwords')

import pandas as pd
from nltk.tokenize import WordPunctTokenizer
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.base import TransformerMixin
import string
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/pedrocondeco/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:

# Transformers
def apply_tokenizer(data, tokenizer):
    lista_tok = [" ".join(tokenizer.tokenize(x)) for x in data]
    return lista_tok

def apply_lowercase(data):
    list_lower = [x.lower() for x in data]
    return list_lower

stopword_list = stopwords.words('english')
def apply_filter_stopwords(data, stopword_list):
    data_no_stopwords = []
    for x in data:
        aux = " ".join([w for w in x.split() if w not in stopword_list])
        data_no_stopwords.append(aux)
    return data_no_stopwords

def apply_filter_punct(data):
    data_no_punct = []
    for x in data:
        
        aux = "".join([w for w in x if w not in string.punctuation])
        data_no_punct.append(aux)

    return data_no_punct

def normalize_whitespace(data):
    return [re.sub(r"^\s+|\s+$|(?<=\s)\s*", "", text) for text in data]

def apply_stemmer(data, stemmer):
    list_tok = [WordPunctTokenizer().tokenize(x) for x in data]
    stems = [" ".join(list(map(stemmer.stem, y))) for y in list_tok]
    return stems

# creates a new column helpful_count/rates_count
class TransformLabels(TransformerMixin):
    def __init__(self, threshold = .5):
        self.helpful_count = 'helpful_count'
        self.rates_count = 'rates_count'
        self.threshold = threshold
        
    def isHelpful(self, helpful_count, rates_count):
        if rates_count == 0 or rates_count < 5:
            return 5 # we don't know, not helpfull for now
        if helpful_count > rates_count:
            return 5 # five means bad, needs to be droped
        val = helpful_count/rates_count
        if val >= self.threshold:
            return 1
        return 0

    def transform(self, X, *_):
        _X = X.copy()

        label_array = []
        for h, r in zip(_X[self.helpful_count], _X[self.rates_count]):
            is_helpul = self.isHelpful(h,r)
            label_array += [is_helpul]

        out = pd.Series(label_array)
        return out
    
    def fit(self, *_):
        return self

class TextTransformer(TransformerMixin):
    def __init__(self, key):
        self.key = key

    def transform(self, X, *_):
        _X = X.copy()
        words = _X[self.key]
        features =  TfidfTransformer().fit_transform(words)
        return features
    
    def fit(self, *_):
        return self

class LazyTranformer(TransformerMixin):
    def __init__(self, key):
        self.key = key

    def transform(self, X, *_):
        _X = X.copy()
        return _X[self.key].to_frame()
    
    def fit(self, *_):
        return self

class Selector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        
    def fit(self, X, y=None):
        return self
    

class TextSelector(Selector):
    def transform(self, X):
        return X[self.key]

class NumberSelector(Selector):
    def transform(self, X):
        return X[[self.key]]

class TextCleanerTransformer(TransformerMixin):
    def __init__(self, column_key, tokenizer, lower=True, remove_punct=True, stopwords=[], stemmer=None):
        self.column_key = column_key
        self.tokenizer = tokenizer
        self.stemmer = stemmer
        self.lower = lower
        self.remove_punct = remove_punct
        self.stopwords = stopwords
    
    def clean_sentences(self, data):
                
        # Split sentence into list of words
        sentences_preprocessed = apply_tokenizer(data, self.tokenizer)
        # YOUR CODE HERE
        #raise NotImplementedError()
        
        # Lowercase
        if self.lower:
            sentences_preprocessed = apply_lowercase(sentences_preprocessed)
            # YOUR CODE HERE
            #raise NotImplementedError()

        if self.stopwords:
            sentences_preprocessed = apply_filter_stopwords(sentences_preprocessed,self.stopwords)
            # YOUR CODE HERE
            #raise NotImplementedError()
            
        # Remove punctuation
        if self.remove_punct:
            sentences_preprocessed = apply_filter_punct(sentences_preprocessed)
            # YOUR CODE HERE
            #raise NotImplementedError()
        
        # Normalize whitespace
        sentences_preprocessed = normalize_whitespace(sentences_preprocessed)
        # YOUR CODE HERE
        #raise NotImplementedError()
    
        # Stem words
        if self.stemmer:
            sentences_preprocessed = apply_stemmer(sentences_preprocessed,self.stemmer)
            # YOUR CODE HERE
            #raise NotImplementedError()

        return sentences_preprocessed

    def transform(self, X, *_):
        _X = X.copy()
        column_to_transform = _X[self.column_key]
        return self.clean_sentences(column_to_transform)


    def fit(self, X, y=None):
        return self

In [23]:
# average word size 
def get_av_word_size(docs):
    av_words_size_per_doc = [] 
    for doc in docs: 
        words_size = [len(word) for word in doc.split()]
        av_size = np.mean(np.array(words_size))
        av_words_size_per_doc.append(av_size)
    return av_words_size_per_doc
# get_av_word_size(docs)  

# number of words 
def get_nb_words(docs): 
    return ([len(doc.split()) for doc in docs])

# get_nb_words(docs)

# number of unique words (proxy for how rich is the reviewer vocabulary)
# not really sure if this feature will be useful (maybe couple with PoS) 
def get_nb_unique_words(docs):
    unique_words = [len(set(word for word in doc)) for doc in docs] 
    normalized = (np.array(unique_words)/np.array(get_nb_words(docs))).tolist()
    return normalized
# get_nb_unique_words(docs)

# getting NERs (proxy for attention to detail by reviewers that describe book parts, e.g. naming book charters)
def get_most_common_persons(nlp_docs, n_most_common=10): 
    matcher = Matcher(nlp.vocab)
    pattern = [[{'ENT_TYPE':'PERSON'}]]
    matcher.add('ppl', pattern)

    persons = []
    for doc in nlp_docs: 
        matches = matcher(doc)
        for match_id, start, end in matches: 
            persons.append(str(doc[start:end]))

    most_common_ents = [person for person in Counter(persons).most_common()[:n_most_common]]
    return most_common_ents

# get_most_common_persons(nlp_docs)    

# number of predefined parts of speech 
def get_nb_PoS(nlp_docs): # docs tokenized string  
    PoS_list = ['ADJ','NOUN','VERB'] 
    nb_PoS = [] 
    for doc in nlp_docs: 
        total_PoS = [token.pos_ for token in doc if token.pos_ in PoS_list]  
        nb_in_doc = sum([Counter(total_PoS)[x] for x in PoS_list])
        nb_PoS.append(nb_in_doc)
    return nb_PoS

# get_nb_PoS(nlp_docs)

def get_nb_unique_PoS(nlp_docs): # docs tokenized string  
    PoS_list = ['ADJ','NOUN'] 
    nb_unique_PoS = [] 
    nb_PoS = []
    word4PoS = [] 
    for doc in nlp_docs: 
        total_PoS = [token.pos_ for token in doc]
        mask = np.array([True if PoS in PoS_list else False for PoS in total_PoS])
        # print(mask[:10])
        words = np.array([token.text for token in doc])
        words_under_selected_PoS = words[mask].tolist()
        word4PoS.append(words_under_selected_PoS)
        # print(words_under_selected_PoS)
        unique_words = get_nb_unique_words(words_under_selected_PoS)
        nb_unique_PoS.append(len(unique_words))
        nb_PoS.append(len(words_under_selected_PoS))
        
    ratio = np.array(nb_unique_PoS)/ np.array(nb_PoS)
    return ratio.tolist()
    
# get_nb_unique_PoS(nlp_docs)

def add_extra_features(df, someString,  docs, nlp_docs): 
    df[someString+"nb_words"] = pd.Series(get_nb_words(docs)).apply(lambda x: 0 if x == None or x == np.NaN  else x)
    df[someString+"av_word_size"] = pd.Series(get_av_word_size(docs)).apply(lambda x: 0 if x == None or x == np.NaN  else x)
    df[someString+"nb_unique_words"] = pd.Series(get_nb_unique_words(docs)).apply(lambda x: 0 if x == None or x == np.NaN  else x)
    
    
    # get_most_common_persons(nlp_docs, n_most_common=10) 
    
    df[someString+"nb_PoS"] = pd.Series(get_nb_PoS(nlp_docs)).apply(lambda x: 0 if x == None or x == np.NaN  else x)
    #df["nb_unique_PoS"] = pd.Series(get_nb_unique_PoS(nlp_docs))

    #df[someString+"nb_words"] = df[someString+"nb_words"].fillna(0, inplace=True)
    #df[someString+"av_word_size"] = df[someString+"av_word_size"].fillna(0, inplace=True)
    #df[someString+"nb_unique_words"] = df[someString+"nb_unique_words"].fillna(0, inplace=True)
    #df[someString+"nb_PoS"] = df[someString+"nb_PoS"].fillna(0, inplace=True)

    return df

In [5]:
# configuration
#data_set = pc.getFile()  # gets path of tfile, you may replace with a string
data_set="/home/pedrocondeco/ldsa/group3/data/book_review_labelled_data.csv"


In [6]:
# configuration
#data_set_to_generalize = pc.getFile()  # gets path of tfile, you may replace with a string
data_set_to_generalize="/home/pedrocondeco/ldsa/group3/data/book_review_test_data_unlabelled.csv"

In [7]:
# configuration
label = "label" # TODO

In [8]:
# Load the dataset
df = pd.read_csv(data_set) 
df['reviewText'] = text_cleaner = TextCleanerTransformer(
    "reviewText",
    WordPunctTokenizer(),
    lower=True, 
    remove_punct=True, 
    stopwords=stopwords.words('english'),
    stemmer=SnowballStemmer("english"),
).transform(df)
df['summary'] = text_cleaner = TextCleanerTransformer(
    "summary",
    WordPunctTokenizer(),
    lower=True, 
    remove_punct=True, 
    stopwords=stopwords.words('english'),
    stemmer=SnowballStemmer("english"),
).transform(df)

# Load the stuff to generalize
df_to_generalize = pd.read_csv(data_set_to_generalize) 
df_to_generalize['reviewText'] = text_cleaner = TextCleanerTransformer(
    "reviewText",
    WordPunctTokenizer(),
    lower=True, 
    remove_punct=True, 
    stopwords=stopwords.words('english'),
    stemmer=SnowballStemmer("english"),
).transform(df_to_generalize)
df_to_generalize['summary'] = text_cleaner = TextCleanerTransformer(
    "summary",
    WordPunctTokenizer(),
    lower=True, 
    remove_punct=True, 
    stopwords=stopwords.words('english'),
    stemmer=SnowballStemmer("english"),
).transform(df_to_generalize)



# Show basic info
print(df.dtypes)            # all columns/types
print("Shape: ",df.shape)   # shape
df.head(10)

reviewerID       object
reviewerName     object
reviewText       object
overall           int64
summary          object
reviewTime       object
rates_count       int64
helpful_count     int64
rating            int64
dtype: object
Shape:  (49992, 9)


,reviewerID,reviewerName,reviewText,overall,summary,reviewTime,rates_count,helpful_count,rating
0,A3UPFTGAWZ3G2R,David J. Loftus,jenkin histori professor member parliament wel...,4,quit readabl nice done,"12 6, 2001",40,37,4
1,A1XTKTLNSCRLDS,Ellen Rappaport,detect inspector erlendur sveinsson best uncov...,5,mesmer depth,"02 23, 2014",0,0,5
2,A1A77B6DQQH436,"crescamp ""esc""",read purchas gift famili small children hope m...,3,10 minut life lesson kid,"02 12, 2013",3,0,3
3,AEAF4MRYHJZI,"Angelia Menchan ""acvermen.blogspot.com""",fierc angel sheri park read like dissert afric...,4,fierc,"03 24, 2010",9,9,4
4,A3B7KU72LGWFER,"Grifel ""Tea Time""",clear author two goal mind 1 take advantag ame...,1,drivel,"06 21, 2003",19,13,1
5,A3JD07VHDLT5FF,"isala ""Isabel and Lars""",collect stori memori japanes soldier fought bu...,5,compel stori ordinari peopl,"03 19, 2005",7,5,5
6,A1JBVAMC9YU0WD,ED,glad borrow book librari instead part hard ear...,1,chees stink,"09 25, 2000",6,5,1
7,A1O7OXC13G9X3E,"kayp75 ""kayp75""",realli enjoy put charact describ beauti felt c...,5,great read,"10 21, 2013",0,0,5
8,A1RAUVCWYHTQI4,A. Ross,read lot scienc fiction mayb five novel year t...,3,lot good element fail add engag read,"08 4, 2010",3,2,3
9,A3PZTH1DTX6O6B,"H. F. Gibbard ""History Buff""",legendari pornograph larri flynt team historia...,4,fun occasion debat histori presidenti sex,"06 11, 2011",5,4,4


In [9]:
# Add label to df (without this it's not possible to train the model)
df[label] = TransformLabels(threshold=.75).fit_transform(df)  
# Show label distribution
pc.showColumnDistribution(df,label)

Number of entries:  49992

Distribution of the column:


5    39420
1     5862
0     4710
Name: label, dtype: int64


Nan Values:  0


In [10]:
# Drop bad stuff?
df = df[df[label] != 5]
pc.showColumnDistribution(df,label)

Number of entries:  10572

Distribution of the column:


1    5862
0    4710
Name: label, dtype: int64


Nan Values:  0


In [11]:
df.head()

,reviewerID,reviewerName,reviewText,overall,summary,reviewTime,rates_count,helpful_count,rating,label
0,A3UPFTGAWZ3G2R,David J. Loftus,jenkin histori professor member parliament wel...,4,quit readabl nice done,"12 6, 2001",40,37,4,1
3,AEAF4MRYHJZI,"Angelia Menchan ""acvermen.blogspot.com""",fierc angel sheri park read like dissert afric...,4,fierc,"03 24, 2010",9,9,4,1
4,A3B7KU72LGWFER,"Grifel ""Tea Time""",clear author two goal mind 1 take advantag ame...,1,drivel,"06 21, 2003",19,13,1,0
5,A3JD07VHDLT5FF,"isala ""Isabel and Lars""",collect stori memori japanes soldier fought bu...,5,compel stori ordinari peopl,"03 19, 2005",7,5,5,0
6,A1JBVAMC9YU0WD,ED,glad borrow book librari instead part hard ear...,1,chees stink,"09 25, 2000",6,5,1,1


In [12]:
# df
docs_reviewText = df['reviewText'].tolist()
docs_summary = df['summary'].tolist()

# df to generalize 
docs_reviewText_generalize = df_to_generalize['reviewText'].tolist()
docs_summary_generalize = df_to_generalize['summary'].tolist()

nlp = spacy.load('en_core_web_sm')

In [20]:
# df
nlp_docs__docs_reviewText = list(tqdm(nlp.pipe(docs_reviewText), total=len(docs_reviewText)))
nlp_docs__docs_summary = list(tqdm(nlp.pipe(docs_summary), total=len(docs_summary)))

# df to generalize 
nlp_docs__docs_reviewText_generalize = list(tqdm(nlp.pipe(docs_reviewText_generalize), total=len(docs_reviewText_generalize)))
nlp_docs__docs_summary_generalize = list(tqdm(nlp.pipe(docs_summary_generalize), total=len(docs_summary_generalize)))



100%|██████████| 10572/10572 [00:03<00:00, 2659.30it/s]


In [24]:
# df
add_extra_features(df, "summary", docs_summary, nlp_docs__docs_summary)
add_extra_features(df, "reviewText", docs_reviewText, nlp_docs__docs_reviewText)

# df to generalize 
add_extra_features(df_to_generalize, "summary", docs_summary_generalize, nlp_docs__docs_summary_generalize)
add_extra_features(df_to_generalize, "reviewText", docs_reviewText_generalize, nlp_docs__docs_reviewText_generalize)

/home/pedrocondeco/.local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3420: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/pedrocondeco/.local/lib/python3.7/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/pedrocondeco/.local/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide


,reviewerID,reviewerName,reviewText,summary,reviewTime,rating,summarynb_words,summaryav_word_size,summarynb_unique_words,summarynb_PoS,reviewTextnb_words,reviewTextav_word_size,reviewTextnb_unique_words,reviewTextnb_PoS
0,A2HESNQJZ9OB7H,Jen,bore stupid hard time finish plot close discri...,unbeliev,"02 16, 2014",1,1,8.000000,7.000000,1,53,4.509434,0.433962,47
1,A1ABXPSFA9PC8N,Ben Parker,ill first admit best cook download book want s...,easi clear cook,"11 7, 2012",5,3,4.333333,3.333333,3,40,4.925000,0.600000,31
2,AYVW3O6W8S5S4,Johnny in Texas,tell anyth spotlight home peopl cobbl togeth l...,bad,"02 25, 2014",3,1,3.000000,3.000000,1,15,5.000000,1.466667,9
3,A17GOTFSUAWN17,Tyson,whenev met last two girlfriend person script s...,short book good primer text girl,"03 15, 2014",4,6,4.500000,2.666667,6,121,4.983471,0.231405,91
4,A2VO8K861AV83R,"Avatheps ""Avatheps""",read review decid take chanc collect seem offe...,disappoint could finish,"12 30, 2013",2,3,7.000000,4.666667,2,91,5.076923,0.307692,66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,A3SIU6DPNBMRIS,Torben Capiau,book written well stori easi understand enjoy ...,sweet book,"02 19, 2014",5,2,4.500000,4.000000,2,32,5.062500,0.718750,25
1996,A201Y92H6J38KS,"Mariekris D. Silva ""little bookworm""",expect complet stori sore disappont stori nice...,happen,"05 6, 2013",3,1,6.000000,5.000000,1,16,5.437500,1.187500,12
1997,A1Y9H69VZQY6BY,Matthew S Blais,littl predict good light read charactersar wel...,good stori,"04 18, 2013",4,2,4.500000,4.000000,2,14,5.500000,1.357143,12
1998,A1IK5W2W4AFRZ3,"The BookWorm ""Reader""",book take place expect go great write great ch...,wow great book,"03 28, 2013",5,3,4.000000,3.333333,2,58,5.051724,0.396552,46


In [25]:
df.fillna(0, inplace=True)
df_to_generalize.fillna(0, inplace=True)


In [27]:
df_to_generalize

,reviewerID,reviewerName,reviewText,summary,reviewTime,rating,summarynb_words,summaryav_word_size,summarynb_unique_words,summarynb_PoS,reviewTextnb_words,reviewTextav_word_size,reviewTextnb_unique_words,reviewTextnb_PoS
0,A2HESNQJZ9OB7H,Jen,bore stupid hard time finish plot close discri...,unbeliev,"02 16, 2014",1,1,8.000000,7.000000,1,53,4.509434,0.433962,47
1,A1ABXPSFA9PC8N,Ben Parker,ill first admit best cook download book want s...,easi clear cook,"11 7, 2012",5,3,4.333333,3.333333,3,40,4.925000,0.600000,31
2,AYVW3O6W8S5S4,Johnny in Texas,tell anyth spotlight home peopl cobbl togeth l...,bad,"02 25, 2014",3,1,3.000000,3.000000,1,15,5.000000,1.466667,9
3,A17GOTFSUAWN17,Tyson,whenev met last two girlfriend person script s...,short book good primer text girl,"03 15, 2014",4,6,4.500000,2.666667,6,121,4.983471,0.231405,91
4,A2VO8K861AV83R,"Avatheps ""Avatheps""",read review decid take chanc collect seem offe...,disappoint could finish,"12 30, 2013",2,3,7.000000,4.666667,2,91,5.076923,0.307692,66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,A3SIU6DPNBMRIS,Torben Capiau,book written well stori easi understand enjoy ...,sweet book,"02 19, 2014",5,2,4.500000,4.000000,2,32,5.062500,0.718750,25
1996,A201Y92H6J38KS,"Mariekris D. Silva ""little bookworm""",expect complet stori sore disappont stori nice...,happen,"05 6, 2013",3,1,6.000000,5.000000,1,16,5.437500,1.187500,12
1997,A1Y9H69VZQY6BY,Matthew S Blais,littl predict good light read charactersar wel...,good stori,"04 18, 2013",4,2,4.500000,4.000000,2,14,5.500000,1.357143,12
1998,A1IK5W2W4AFRZ3,"The BookWorm ""Reader""",book take place expect go great write great ch...,wow great book,"03 28, 2013",5,3,4.000000,3.333333,2,58,5.051724,0.396552,46


In [36]:
# Split
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
train_label = df_train['label']
df_train.drop(label, inplace=True, axis=1)   # labels MUST NOT exist within train set

/home/pedrocondeco/.local/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [29]:

# Define features here, keep standard, concatenate _feature in the end
review_text_feature = Pipeline([
                                ('review_text_selector', TextSelector("reviewText")),
                                ('tfidf', TfidfVectorizer())
                            ])
summary_feature = Pipeline([
                                ('summary_selector', TextSelector("summary")),
                                ('tfidf', TfidfVectorizer())
                            ])
rating_feature = Pipeline([
                                ('rating_selector', NumberSelector("rating")),
                                ('onehot', OneHotEncoder())
                            ])
summarynb_words = Pipeline([
                                ('summarynb_words', NumberSelector("summarynb_words")),
                                ('standardScaler', StandardScaler())
                            ])
summaryav_word_size = Pipeline([
                                ('summaryav_word_size', NumberSelector("summaryav_word_size")),
                                ('standardScaler', StandardScaler())
                            ])                           
summarynb_unique_words = Pipeline([
                                ('summarynb_unique_words', NumberSelector("summarynb_unique_words")),
                                ('standardScaler', StandardScaler())
                            ])                           
summarynb_PoS = Pipeline([
                                ('summarynb_PoS', NumberSelector("summarynb_PoS")),
                                ('standardScaler', StandardScaler())
                            ])  





reviewTextnb_words = Pipeline([
                                ('reviewTextnb_words', NumberSelector("reviewTextnb_words")),
                                ('standardScaler', StandardScaler())
                            ])
reviewTextav_word_size = Pipeline([
                                ('reviewTextav_word_size', NumberSelector("reviewTextav_word_size")),
                                ('standardScaler', StandardScaler())
                            ])                           
reviewTextnb_unique_words = Pipeline([
                                ('reviewTextnb_unique_words', NumberSelector("reviewTextnb_unique_words")),
                                ('standardScaler', StandardScaler())
                            ])                           
reviewTextnb_PoS = Pipeline([
                                ('reviewTextnb_PoS', NumberSelector("reviewTextnb_PoS")),
                                ('standardScaler', StandardScaler())
                            ])  


# Features
features = FeatureUnion([
                        ('summary_feature',summary_feature),
                        ('review_text_feature',review_text_feature),
                        ('rating_feature', rating_feature),
                        ('summarynb_words',summarynb_words),
                        ('summaryav_word_size',summaryav_word_size),
                        ('summarynb_unique_words',summarynb_unique_words),
                        ('summarynb_PoS',summarynb_PoS),
                        ('reviewTextnb_words',reviewTextnb_words),
                        ('reviewTextav_word_size',reviewTextav_word_size),
                        ('reviewTextnb_unique_words',reviewTextnb_unique_words),
                        ('reviewTextnb_PoS',reviewTextnb_PoS),
                        ])

In [30]:
ok = features.fit_transform(df_train)
print(ok.todense().shape)

(10043, 48495)


In [43]:
# Pipeline w/ model
model_pipeline = Pipeline([
                        ('features', features),
                        ('model',RandomForestClassifier(random_state = 42))
                    ])

In [44]:
# Baseline and print silly values
# returns model
def baseline(train_set, train_labels, test_set, model):
  # Train the classifier
  model.fit(train_set, train_labels)

  # Predict
  model_predictions = model.predict(test_set)

  # Print F1 stuff
  print(classification_report(test_set[label], model_predictions))

  _acc = np.mean(model_predictions == test_set[label])
  print('Accuracy: ' + str(_acc))

  return model

def stupidFunction(x):
    if x == 0:
        return "false"
    return "true"
def predictFromFileAndOutPutCSV(model, df_to_generalize, path):
    predictions = model.predict(df_to_generalize)
    out = pd.DataFrame({'is_helpful': predictions})
    out['is_helpful'] =  out['is_helpful'].apply(lambda x: stupidFunction(x))
    out.to_csv(path, index=False)


In [45]:
model = baseline(df_train, train_label, df_test, model_pipeline)

              precision    recall  f1-score   support

           0       0.49      0.57      0.53       933
           1       0.61      0.53      0.57      1182

    accuracy                           0.55      2115
   macro avg       0.55      0.55      0.55      2115
weighted avg       0.56      0.55      0.55      2115

Accuracy: 0.548936170212766


In [ ]:
df_to_generalize

In [35]:
predictFromFileAndOutPutCSV(model, df_to_generalize,'./quinta.csv')